# Clean

In [1]:
%%time
from plotly import graph_objects as go
import plotly.express as px
import pandas as pd, numpy as np, datetime as dt, json, math, pickle
from scipy.optimize import curve_fit
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', 9999)
ts_str = '2020_09_15_15_18_23'
df = pd.read_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', parse_dates=True)
print(df.shape, ', '.join(df.columns))
def power_law(x, a, b): # Estimation Model: ln(y) = a+b*x
    return a+b*x
def power_law_inve(x, a, b):
    return np.power(np.e, a+b*x)
def power_law_diff(x, a, b):
    return (a+b*x)*np.power(np.e, (a+b*x)-1)

(285054, 19) SAPCode, SiteName, WorkCentreName, Sloc, Day, Time, SupplierName, PlateLicense, TruckType, Ticket_no, Size_inch, InboundWeight, MillWeight, OutboundWeight, Purchase_price_thb_ton, Price_ton, Month, Year, TransactionType
Wall time: 1.47 s


In [2]:
#print(df.TransactionType.value_counts())
df = df.loc[df.TransactionType=='ชั่งซื้อ']; print(df.shape, 1)
df = df.loc[df.SupplierName.notnull()]; print(df.shape, 2)
df = df.loc[df.MillWeight>0]; print(df.shape, 3)
df = df.loc[(df.Price_ton>500)]; print(df.shape, 4) # &(df.Price_ton<1500)
df = df.loc[df.Price_ton>0]; print(df.shape, 5)
#df = df.loc[df.Size_inch=='ยูคาไม้ท่อน 2.0 นิ้ว']; print(df.shape, 6)

(284961, 19) 1
(284959, 19) 2
(284905, 19) 3
(260365, 19) 4
(260365, 19) 5


In [3]:
df = df.loc[~df.Sloc.isin(['W1NS','W1CP','N1BK'])]
df = df.loc[~df.SiteName.isin(['สามง่าม','วังวน','บางกระทุ่ม','ตาก', 'CC6'])]
print(df.shape)
df.loc[df.SiteName=='โรงสับไม้กำแพงเพชร', 'Sloc'] = 'CH11'
df.loc[df.SiteName=='โรงสับไม้ชุมพวง', 'Sloc'] = 'CH12'
df.loc[df.SiteName=='โรงสับไม้พระยืน', 'Sloc'] = 'CH13'
df['Sloc'] = df.Sloc.astype('category')
df['Sloc'] = df.Sloc.cat.as_ordered()
sloc = dict(df.groupby('SiteName')['Sloc'].max())
sloc = dict([(value, key) for key, value in sloc.items()])
df['Sloc'] = df.Sloc.astype('str')
df['Zone'] = df['Sloc'].str[:2]
df['Zone'] = df['Sloc'].astype('category')
zone = df.groupby('Zone').agg({'Sloc': lambda x: list(np.unique(x))})['Sloc'].to_dict()

(259957, 19)


In [4]:
ck = ['Zone', 'Sloc','Year', 'Month', 'Day', 'MillWeight', 'Price_ton', 'Purchase_price_thb_ton', 'SupplierName']
df = df.drop(df.loc[(df.Year == dt.datetime.now().year) & (df.Month == dt.datetime.now().month)].index)[ck]
df = df.loc[df.Year>=2016].reset_index(drop=True)
print(df.shape) # WorkCentreName, 'Time', 'Ticket_no', SiteName
df['Year'] = df.Year.astype('str') # Change year type to string
#df['Year'] = df.Year.cat.as_ordered()
year = dt.datetime.now().year
years = [str(year) for year in range(year-4,year+1)]

(233923, 9)


# Supply

In [5]:
supply = df.copy()
supply = supply.groupby(['Sloc','Year','Month','Price_ton']).agg({'MillWeight':'sum', 'Purchase_price_thb_ton':'sum', 'SupplierName':'nunique', 'Day':'count'}).reset_index()
supply = supply.rename(columns={'Day':'Num_Tran'}).sort_values(by=['Year','Month','Price_ton']).reset_index(drop=True) # , ascending=False

In [6]:
%%time
supply_= supply.copy()
supply = {}
for s in sloc:
    supply[s] = {}
    for y in years:
        supply[s][y] = {}
        for m in range(1,13):
            if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
                supply[s][y][m] = supply_.loc[(supply_.Sloc==s)&(supply_.Year==y)&(supply_.Month==m)]
                supply[s][y][m] = supply[s][y][m].merge(supply[s][y][m][['MillWeight', 'Purchase_price_thb_ton']].cumsum(),
                                                        left_index=True, right_index=True, suffixes=('','_c'))
                supply[s][y][m]['Type'] = 'Actual'
                supply[s][y][m]['Sloc'] = s

Wall time: 3.45 s


In [7]:
s_2020_8 = ['CH11', 'CH13', 'N2KP', 'N3CD', 'N3TK', 'N1TN', 'N3NB', 'N1TC', 'N1NK', 'CH12']
s_2020_8_= supply_.loc[(supply_.Year=='2020')&(supply_.Month==8)&(supply_.Sloc.isin(s_2020_8))]

In [8]:
f"{s_2020_8_.MillWeight.sum():,.2f}"

'9,899.84'

In [9]:
s_2020_8_.groupby('Sloc')['MillWeight'].sum()

Sloc
CH11   3,692.0700
CH12   4,107.8300
CH13   1,222.8900
N1NK      49.5000
N1TC      19.1000
N1TN      15.8300
N2KP     196.3900
N3CD     320.5100
N3NB     149.4200
N3TK     126.3000
Name: MillWeight, dtype: float64

# Curve Fitting

In [10]:
sl, ml = {}, {}
for s in sloc:
    sl[s], ml[s] = {}, {}
    for m in range(1,13):
        sl[s][m], ml[s][m] = [], []

In [11]:
%%time
pars, cov, spm = {}, {}, {}; i = 0
for s in sloc: #['CH11']:#
    pars[s], cov[s], spm[s] = {}, {}, {}
    for y in years:
        pars[s][y], cov[s][y], spm[s][y] = {}, {}, {}
        for m in range(1,13):
            spm[s][y][m] = pd.DataFrame(); i += 1
            print(f'{str(i).zfill(4)}-{s}-{y}-{str(m).zfill(2)}', end=' ')
            if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
                if len(supply[s][y][m]) > 2:
                    print(0, end='')
                    xdata = supply[s][y][m].MillWeight_c
                    ydata = supply[s][y][m].Price_ton
                    xmax  = xdata.max()
                    sl[s][m] = sl[s][m] + [y]
                    ml[s][m] = ml[s][m] + [xmax]
                    pars[s][y][m], cov[s][y][m] = curve_fit(f=power_law, bounds=((-np.inf, 0), np.inf), xdata=xdata, ydata=np.log(ydata))
                    mm    = range(1,round(xmax*10),10)
                    yhat  = power_law_inve(mm, *pars[s][y][m])
                    sen   = power_law_diff(mm, *pars[s][y][m])
                    supply[s][y][m]  = supply[s][y][m].append(pd.DataFrame({'Type': 'Estimation','Sloc': s,'Year': y,
                                                                            'Price_ton': yhat, 'MillWeight_c': mm}))
                    spm[s][y][m]  = spm[s][y][m].append(pd.DataFrame({'Sloc': s, 'Year': y, 'Month': m,
                                                                      'Sensitivity': sen, 'Price_ton':yhat, 'MillWeight_c': mm}))
            print('')
        print('')
    print('')
# stdevs = np.sqrt(np.diag(cov))
#supply[s][y][m]['Price_ton_r'] = supply[s][y][m].Price_ton - power_law(supply[s][y][m].MillWeight_c, *pars)

0001-N3CD-2016-01 
0002-N3CD-2016-02 
0003-N3CD-2016-03 
0004-N3CD-2016-04 
0005-N3CD-2016-05 
0006-N3CD-2016-06 
0007-N3CD-2016-07 
0008-N3CD-2016-08 
0009-N3CD-2016-09 
0010-N3CD-2016-10 
0011-N3CD-2016-11 
0012-N3CD-2016-12 

0013-N3CD-2017-01 
0014-N3CD-2017-02 
0015-N3CD-2017-03 
0016-N3CD-2017-04 
0017-N3CD-2017-05 
0018-N3CD-2017-06 
0019-N3CD-2017-07 
0020-N3CD-2017-08 
0021-N3CD-2017-09 
0022-N3CD-2017-10 
0023-N3CD-2017-11 
0024-N3CD-2017-12 

0025-N3CD-2018-01 
0026-N3CD-2018-02 
0027-N3CD-2018-03 
0028-N3CD-2018-04 
0029-N3CD-2018-05 
0030-N3CD-2018-06 
0031-N3CD-2018-07 
0032-N3CD-2018-08 
0033-N3CD-2018-09 
0034-N3CD-2018-10 
0035-N3CD-2018-11 
0036-N3CD-2018-12 

0037-N3CD-2019-01 
0038-N3CD-2019-02 0
0039-N3CD-2019-03 0
0040-N3CD-2019-04 0
0041-N3CD-2019-05 0
0042-N3CD-2019-06 0
0043-N3CD-2019-07 0
0044-N3CD-2019-08 
0045-N3CD-2019-09 
0046-N3CD-2019-10 
0047-N3CD-2019-11 
0048-N3CD-2019-12 

0049-N3CD-2020-01 0
0050-N3CD-2020-02 0
0051-N3CD-2020-03 0
0052-N3CD-2020-04 

0517-N2ST-2019-01 0
0518-N2ST-2019-02 0
0519-N2ST-2019-03 
0520-N2ST-2019-04 
0521-N2ST-2019-05 0
0522-N2ST-2019-06 
0523-N2ST-2019-07 0
0524-N2ST-2019-08 
0525-N2ST-2019-09 
0526-N2ST-2019-10 
0527-N2ST-2019-11 
0528-N2ST-2019-12 

0529-N2ST-2020-01 
0530-N2ST-2020-02 
0531-N2ST-2020-03 0
0532-N2ST-2020-04 0
0533-N2ST-2020-05 0
0534-N2ST-2020-06 
0535-N2ST-2020-07 
0536-N2ST-2020-08 
0537-N2ST-2020-09 
0538-N2ST-2020-10 
0539-N2ST-2020-11 
0540-N2ST-2020-12 


0541-N1PR-2016-01 
0542-N1PR-2016-02 
0543-N1PR-2016-03 
0544-N1PR-2016-04 
0545-N1PR-2016-05 
0546-N1PR-2016-06 
0547-N1PR-2016-07 
0548-N1PR-2016-08 
0549-N1PR-2016-09 
0550-N1PR-2016-10 
0551-N1PR-2016-11 
0552-N1PR-2016-12 

0553-N1PR-2017-01 
0554-N1PR-2017-02 
0555-N1PR-2017-03 
0556-N1PR-2017-04 
0557-N1PR-2017-05 
0558-N1PR-2017-06 
0559-N1PR-2017-07 
0560-N1PR-2017-08 
0561-N1PR-2017-09 
0562-N1PR-2017-10 
0563-N1PR-2017-11 
0564-N1PR-2017-12 

0565-N1PR-2018-01 
0566-N1PR-2018-02 
0567-N1PR-2018-03 
0568-N1PR-2018-04 


1059-N2SB-2019-03 0
1060-N2SB-2019-04 0
1061-N2SB-2019-05 0
1062-N2SB-2019-06 0
1063-N2SB-2019-07 0
1064-N2SB-2019-08 0
1065-N2SB-2019-09 
1066-N2SB-2019-10 0
1067-N2SB-2019-11 
1068-N2SB-2019-12 0

1069-N2SB-2020-01 0
1070-N2SB-2020-02 0
1071-N2SB-2020-03 0
1072-N2SB-2020-04 0
1073-N2SB-2020-05 0
1074-N2SB-2020-06 
1075-N2SB-2020-07 
1076-N2SB-2020-08 
1077-N2SB-2020-09 
1078-N2SB-2020-10 
1079-N2SB-2020-11 
1080-N2SB-2020-12 


1081-W1SR-2016-01 
1082-W1SR-2016-02 
1083-W1SR-2016-03 
1084-W1SR-2016-04 
1085-W1SR-2016-05 
1086-W1SR-2016-06 
1087-W1SR-2016-07 
1088-W1SR-2016-08 
1089-W1SR-2016-09 
1090-W1SR-2016-10 
1091-W1SR-2016-11 
1092-W1SR-2016-12 

1093-W1SR-2017-01 
1094-W1SR-2017-02 
1095-W1SR-2017-03 
1096-W1SR-2017-04 
1097-W1SR-2017-05 
1098-W1SR-2017-06 
1099-W1SR-2017-07 
1100-W1SR-2017-08 
1101-W1SR-2017-09 
1102-W1SR-2017-10 
1103-W1SR-2017-11 
1104-W1SR-2017-12 

1105-W1SR-2018-01 
1106-W1SR-2018-02 
1107-W1SR-2018-03 
1108-W1SR-2018-04 
1109-W1SR-2018-05 
1110-W1SR-201

1476-CH13-2018-12 0

1477-CH13-2019-01 0
1478-CH13-2019-02 0
1479-CH13-2019-03 0
1480-CH13-2019-04 0
1481-CH13-2019-05 0
1482-CH13-2019-06 0
1483-CH13-2019-07 0
1484-CH13-2019-08 0
1485-CH13-2019-09 0
1486-CH13-2019-10 0
1487-CH13-2019-11 0
1488-CH13-2019-12 0

1489-CH13-2020-01 0
1490-CH13-2020-02 0
1491-CH13-2020-03 0
1492-CH13-2020-04 0
1493-CH13-2020-05 0
1494-CH13-2020-06 0
1495-CH13-2020-07 0
1496-CH13-2020-08 0
1497-CH13-2020-09 
1498-CH13-2020-10 
1499-CH13-2020-11 
1500-CH13-2020-12 


Wall time: 8.86 s


In [12]:
supply['N3DT']['2018'][8]

,Sloc,Year,Month,Price_ton,MillWeight,Purchase_price_thb_ton,SupplierName,Num_Tran,MillWeight_c,Purchase_price_thb_ton_c,Type
1522,N3DT,2018,8,"1,000.0000","1,442.2700","1,442,270.0000",50,323,"1,442.2700","1,442,270.0000",Actual
1525,N3DT,2018,8,"1,020.0000",3.9900,"4,069.8000",1,1,"1,446.2600","1,446,339.8000",Actual


In [16]:
sen = {}; m=8
for y in years:
    sen[y] = {}
    sen[y][m] = pd.DataFrame()
    for s in sloc: # ['CH11']:#
        if y in sl[s][m]:
            sen_      = supply[s][y][m].loc[(supply[s][y][m].Sloc==s)&(supply[s][y][m].Type.isin(['Estimation', 'Actual']))]
            sen[y][m] = sen[y][m].append(sen_.loc[sen_.MillWeight_c<=30000]) #
px.scatter(sen[y][m], x='MillWeight_c', y='Price_ton', color='Sloc') # Type

In [14]:
px.scatter(sen['2018'][8], x='MillWeight_c', y='Price_ton', color='Sloc')

In [15]:
sen_ = {}; m=1
for y in years:
    sen_[y] = {}
    sen_[y][m] = pd.DataFrame()
    for s in sloc:
        if y in sl[s][m]:
            sen_[y][m] = sen_[y][m].append(spm[s][y][m].loc[spm[s][y][m].MillWeight_c<=30000]) #
px.scatter(sen_[y][m], x='MillWeight_c', y='Sensitivity', color='Sloc') # Type

# Combine

In [ ]:
%%time
spmc, spmcs = {}, {}
for m in range(1,13):
    spmc[m] = {}; spmcs[m] = pd.DataFrame()
    for s in sloc:
        spmc[m][s] = pd.DataFrame()
        for y in years:
            print(s,y,m) # , end=''
            if y in sl[s][m]:
                print(len(spm[s][y][m]['Price_ton']))
                #pty = spm[s][y][m]['Price_ton']
                spmc[m][s] = spmc[m][s].combine_first(spm[s][y][m][['Price_ton','MillWeight_c']].rename(columns={'Price_ton':y,'MillWeight_c':'m_'+y}))
                print(len(spmc[m][s][y]))
            else:
                spmc[m][s][y] = None
                
        spmc[m][s]['Price_ton_avg'] = spmc[m][s][years].mean(axis=1)
        spmc[m][s]['MillWeight_max'] = spmc[m][s][['m_'+y_ for y_ in years if y_ in sl[s][m]]].mean(axis=1)
        spmc[m][s]['Sloc'] = s
        spmc[m][s]['Month'] = m
        #spmc[m][s] = spmc[m][s][['Sloc', 'Month', 'MillWeight_max', 'Price_ton_avg']]
        spmcs[m] = spmcs[m].append(spmc[m][s])

In [ ]:
val_sum = {}
for m in range(1,13):
    val_sum[m] = spmcs[m].sort_values(by='Price_ton_avg').reset_index(drop=True).shift(1).drop(0).reset_index()
    val_sum[m]['val_sum'] = val_sum[m]['Price_ton_avg'].cumsum()

In [ ]:
with open('val_sum.pickle', 'wb') as handle:
    pickle.dump(val_sum, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#val_sum.loc[val_sum.Sloc=='N3DT']

In [ ]:
val_sum[8].iloc[:9920]#.groupby('Sloc')

In [ ]:
px.scatter(val_sum[8], x='index', y='val_sum')

# Actual

In [ ]:
actual, actuals = {}, {}
for y in years:
    actual[y], actuals[y] = {}, {}
    for m in range(1,13):
        actual[y][m] = {}; actuals[y][m] = pd.DataFrame()
        for s in sloc:
            if y in sl[s][m]:
                if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
                    actual[y][m][s] = {}
                    ar = supply[s][y][m].loc[supply[s][y][m].Type=='Actual'].copy()
                    ar['mr'] = ar.MillWeight.apply(lambda x: math.ceil(x))
                    actual[y][m][s] = pd.DataFrame((p, r.Price_ton) for r in ar.itertuples() for p in range(1,r.mr+1)).shift(1).drop(0)
                    actual[y][m][s]['Sloc'] = s
                    actual[y][m][s]['Month'] = m
                actuals[y][m] = actuals[y][m].append(actual[y][m][s])

In [ ]:
print(y,m,s)
actual['2020'][8][s]

In [ ]:
acs = {}
for y in years:
    acs[y] = {}
    for m in range(1,13):
        if y in sl[s][m]:
            if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
                acs[y][m] = actuals[y][m].sort_values(by=1).reset_index(drop=True).shift(1).drop(0).reset_index()
                acs[y][m] = acs[y][m].merge(acs[y][m][[1]].cumsum().rename(columns={1:'Value'}), left_index=True, right_index=True)

In [ ]:
with open('acs.pickle', 'wb') as handle:
    pickle.dump(acs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
acs['2020'][8].Sloc.unique()

In [ ]:
acs['2020'][8].groupby('Sloc').count()

In [ ]:
acs['2020'][8].groupby('Sloc').count().sum()